# Pandas DataFrames (`pd.DataFrame`)

The pandas dataframe is the workhorse of most data science and analytics projects.  The dataframe represents the data you're working with as a table.  However, the flexibility of the dataframe is that each row **and** column is represented as a pandas Series, which allows for many powerful ways to mess around with the data

In [ ]:
import pandas as pd
import numpy as np
import requests

First let's get some data so we can see what we can do with a data frame.  Don't worry about exactly what this function is doing, we will go over it in a bit.

In [ ]:
def get_data(token):
    res = requests.get(
        f'https://api.cryptowat.ch/markets/coinbase-pro/{token}usd/ohlc',
        params={
            'periods': '3600',
            'after': str(int(pd.Timestamp('2021-12-01').timestamp()))
        }
    )

    df = pd.DataFrame(
        res.json()['result']['3600'],
        columns=['ts', 'open', 'high', 'low', 'close', 'volume', 'volumeUSD']
    )
    df['ts'] = pd.to_datetime(df.ts, unit='s')
    df['token'] = token
    
    return df


In [ ]:
tokens = ['BTC', 'ETH', 'SOL', 'AAVE', 'COMP']

Don't worry too much about what is going on in the function below - we'll briefly go over it as it showcases the power of python, but it's not necessary for the class

In [ ]:
dfs = [
    (lambda x: x.assign(chain=np.where(x.token.isin(['ETH', 'AAVE', 'COMP']), np.full(x.shape[0], 'ETH'), x.token)))(get_data(token)) 
    for token in tokens
]

In [ ]:
df_base = pd.concat(get_data(token) for token in tokens)
df_base['chain'] = np.where(df_base.token.isin(['ETH', 'AAVE', 'COMP']), np.full(df_base.shape[0], 'ETH'), df_base.token)


In [ ]:
df = df_base.set_index('ts')

## Understanding the data frame

After loading the data in our data frame, we can now inspect what's inside.  We'll need to do this as often the data we will store will be impossible to inspect row by row, and we will need to check that our data loading was correct

Let's check some basic properties of the data set:

We can see how many rows and columns this data frame has, and total number of data points

In [ ]:
df.shape

In [ ]:
df.size

We can see what the first 5 rows looks like:

In [ ]:
df.head()

...and the last 5 rows

In [ ]:
df.tail()

We can also see a general overview of the schema (column name, data and data type) of the data

In [ ]:
df.info()

as well as descriptive statistics about every column

In [ ]:
df.describe()

## DataFrame Indexing

Indexing in data frames works very similar to Series, however there are now two "axes" that we can operate on - rows and columns.  For example, using `[*]` for indexing (like in series) by default will operate on columns:

In [ ]:
df['open']

however using `.loc[*]` will allow you to access rows:

In [ ]:
df.loc['2021-12-01 01:00:00']

and `.iloc[*]` will get you positional rows

In [ ]:
df.iloc[0]

we can also get to the last row easily

In [ ]:
df.iloc[-1]

or return it as a data frame instead of a Series

In [ ]:
df.iloc[[-1]]

**note**: `df.loc[0]` will not work, as this is accessing via index

---
**note**: Also, the index operators will return a `pd.Series` if there's 1 row returned, or a new `pd.DataFrame` if multiple rows are returned, e.g.:

In [ ]:
type(df.iloc[0])

In [ ]:
type(df.loc['2021-12-01 01:00:00']) # 5 rows returned

we can convert a Series to a DataFrame anytime by using the `.to_frame()` method on the Series object.  This will turn the Series to a DataFrame, using the `Series.name` as the column name

In [ ]:
df.iloc[0].to_frame()

---

In addition, we can select on multiple columns and rows in various ways:

In [ ]:
df[['open', 'close']]

In [ ]:
df[0:2]

In [ ]:
df.loc['2021-12-01 00:00:00':'2021-12-01 02:00:00']

In [ ]:
df.iloc[0:4]

In [ ]:
df.iloc[[0, 4, 10, 50]]

And finally, we can index on both rows and columns at the same time with `.loc`:

In [ ]:
df.loc['2021-12-01 00:00:00':'2021-12-01 02:00:00', ['close', 'volume', 'token']]

**note**: Given that by default dataframe indices are sequential integers by default, it's good practice to use `.loc` and `.iloc` to index into the data frame to be very clear, for example, let's shuffle our data frame then select:

In [ ]:
df_shuffled = df_base.sample(frac=1)

In [ ]:
df_shuffled.loc[[0, 2, 3]]

In [ ]:
df_shuffled.iloc[[0, 2, 3]]

lastly, we can set the DataFrame index from a column, or remove an index into a column

In [ ]:
df_shuffled.set_index('ts')

In [ ]:
df_shuffled.set_index('ts').reset_index()

## DataFrame Filtering

Filtering a data frame is very similar to filtering a series.  We can filter on any set of columns, the filtering is done via indices.  For example, if we wanted to just get the data points for tokens on the ethereum chain:

In [ ]:
df['chain'] == 'ETH'

In [ ]:
df.loc[df['chain'] == 'ETH']

In [ ]:
df.loc[df['chain'] == 'ETH', 'close']

## Deleting from Dataframes

We can select for all the things we'd like, but we can also drop both rows and columns.  This also works by index, i.e.:

In [ ]:
df.drop(pd.to_datetime('2021-12-01 00:00:00'))

In [ ]:
df.drop(columns='volumeUSD')

In [ ]:
df.drop(['close', 'open'], axis=1)

## Common Operations

Like with pandas Series, a DataFrame is simply a numpy array underneath the hood.

In [ ]:
type(df.values)

This means that the operations we saw for pandas Series can be applied to DataFrames as well, e.g. we can apply a scalar to every element in the DataFrame

In [ ]:
df.head() * 10

However, the operation needs to be valid for ALL elements if we want to do this - e.g. while `*` is overridden for strings, `/` is not and will fail

In [ ]:
df.head() / 10

Aggergation functions are by default done by column

In [ ]:
df.mean()

However we can also make them aggregate by row:

In [ ]:
df.mean(axis=1)

## Mutating the Dataframe

Like with other functionality, mutating DataFrames is very similar to mutating Series.  For example, setting one column to a single value is easy:

In [ ]:
df_mutations = df_base.set_index('ts')

In [ ]:
df_mutations.head()

In [ ]:
df_mutations['chain'] = 'NA'

In [ ]:
df_mutations.head()

We can also create a new column and add data by index:

In [ ]:
updates = pd.Series({pd.to_datetime('2021-12-01 00:00:00'): 1})
updates

In [ ]:
df_mutations['start_of_week'] = updates

In [ ]:
df_mutations.head()

We can also use the `.assign(...)` method to update columns, e.g.:

In [ ]:
df_mutations.assign(
    chain=np.where(df_mutations.token.isin(['ETH', 'AAVE', 'COMP']), np.full(df_mutations.shape[0], 'ETH'), df_mutations.token),
    start_of_week=np.NaN
)

**note**: using the index notation `[*]` will mutate the dataframe in place, however `.assign` will return a new data frame

We can also rename columns using a `{from:to}` syntax, e.g.:

In [ ]:
df_mutations.rename(
    columns={
        'open':'OpeningPrice',
        'chain':'CryptoChain'
    }
)

We can also use functions to rename, e.g.:

In [ ]:
df_mutations.rename(columns=lambda x: x.upper())

The above commands will return a new DataFrame.  If we want to rename the input DataFrame, we can use the `inplace` option (which is available on most mutating functions), such as:

In [ ]:
df_mutations.rename(
    columns={
        'open':'OpeningPrice',
        'chain':'CryptoChain'
    },
    inplace=True
)

df_mutations

We can also add rows to the DataFrame by using `append`:

In [ ]:
df_mutations.append(
    pd.Series({
        'high': 1,
        'low': 2,
        'token': 'FAKE'
    }, name=pd.to_datetime('2021-11-30 00:00:00'))
)

## Sorting DataFrames

One thing that we didn't need to really do with Series is sorting.  For DataFrames, we will often need to sort by column(s) or by the index.  We can use `sort_values` and `sort_index` to do this

In [ ]:
df.sort_values('open')

In [ ]:
df.sort_values('open', ascending=False)

In [ ]:
df.sort_values(['volumeUSD', 'open'])

We can also sort by the index

In [ ]:
df.sort_index()

## Grouping DataFrames

one _very common_ action we will do during data manipulation is grouping then aggregating.  Pandas DataFrame has the method `groupby`, which allows us to group by any column in our DataFrame.

`groupby` returns a `DataFrameGroupBy` object, which we can apply a function to each group, or directly aggregate

In [ ]:
df.groupby('chain')

In [ ]:
df.groupby('chain').groups

In [ ]:
len(df.groupby('chain'))

In [ ]:
df.groupby('chain').size()

after grouping, we can operate on the whole DataFrame or on any column

In [ ]:
df.groupby('chain')['volumeUSD'].sum().to_frame()

we can also groupby multiple columns.  The row indices now are a multi-index, however we will not go into this

In [ ]:
df.groupby(['chain', 'token'])['volumeUSD'].sum().to_frame()

We can actually aggregate without setting a compound index by adding `as_index=False`

In [ ]:
df.groupby(['chain', 'token'], as_index=False)['volumeUSD'].sum()

We can now operate on the groups.  For example, if we wanted to sum all columns:

In [ ]:
df.groupby('chain').aggregate(np.sum)

or describe all columns

In [ ]:
df.groupby('chain').describe()

We can also do multiple aggregations

In [ ]:
df.groupby('chain')['open'].agg([np.size, np.mean, np.std, np.min, np.max])

we can actually use _any_ arbitrary functions - for example, we can use lambdas

In [ ]:
df.groupby('chain')['open'].agg(
    range=lambda x: x.max() - x.min()
)

## Joining Dataframes

One of the primary things we need to do before starting to clean data is to make sure that we can get all of our data into one place.  This is usually called either a fat talbe or a long table, depending on how we are doing the joining.  We'll look at a few different ways to join pandas DataFrames below.

We will be using `dfs`, which is a list of DataFrames that we created up above

### `pd.concat`

To join the dataframes lengthwise, we can use `pd.concat`.  This will append the dataframes together, and join the rows by using the column names as an index.  If any dataframe doesn't have a column name another one has, it will appear in the full DataFrame but will have NA for the missing DataFrames

In [ ]:
pd.concat(dfs)

if you wanted to make sure you know where the original data is from, we can add keys, which creates a multi index:

In [ ]:
res = pd.concat(dfs, keys=tokens)
res

this allows us to select the data from the source tables, e.g.:

In [ ]:
res.loc['COMP']

As we saw above, we can also use `.append(*)` on DataFrames as well as Series

In [ ]:
dfs[0].append(dfs[1])

Lastly, remember the importance of indices.  In the operation above (both `.concat` and `.append`) we joined the DataFrames while keeping the indices of the original tables.  This means that we have repeated indices:

In [ ]:
dfs[0].append(dfs[1]).sort_index()

This sometimes isn't ideal, esp. if we want to join against these indices later.  Instead, we can create a new index on the joined table using the `ignore_index` parameter, which allows us to have a sequential, non-repeated index:

In [ ]:
dfs[0].append(dfs[1], ignore_index=True).sort_index()

### `pd.DataFrame.join`

`df.join` is a nice and easy method that allows us to join two dataframes by their index

In [ ]:
dfs[0].set_index('ts')['close'].rename(f'close_{tokens[0]}').to_frame().join(
    dfs[1].set_index('ts')['close'].rename(f'close_{tokens[1]}').to_frame()
)

we can get a little more advanced by having a **left** unkeyed DataFrame joining against a **right** keyed DataFrame, e.g.:

In [ ]:
dfs[0][['ts', 'close']].join(
    dfs[1].set_index('ts')['close'].rename(f'close_{tokens[1]}').to_frame(),
    on='ts'
)

### `pd.merge`

`pd.merge` is Pandas way of doing sql-like joins (e.g. left join, inner join, outer join etc).  There are a few quirks we'll see though.

In [ ]:
pd.merge(
    dfs[0][['ts', 'close']].rename(columns={'close': f'close_{tokens[0]}'}),
    dfs[1][['ts', 'close']].rename(columns={'close': f'close_{tokens[1]}'}),
    on='ts',
    how='inner'
)

we can use other conditions for `how`, e.g. 'left', 'right', 'outer', and 'cross'

if left and right DataFrames have columns with the same name, pandas will automatically resolve the delta by adding `_x` and `_y` suffixes to the conflicted columns

In [ ]:
pd.merge(
    dfs[0][['ts', 'close']],
    dfs[1][['ts', 'close']],
    on='ts',
    how='inner'
)

however, we can also define our own suffixes, e.g.

In [ ]:
pd.merge(
    dfs[0][['ts', 'close']],
    dfs[1][['ts', 'close']],
    on='ts',
    how='inner',
    suffixes=[f'_{tokens[0]}', f'_{tokens[1]}']
)